### Generating noun phrases from SciDocs for query classification

- SciDocs uses MAG to generate a dataset with level:1 classes

`
0	Art
1	Biology
2	Business
3	Chemistry
4	Computer science
5	Economics
6	Engineering
7	Environmental science
8	Geography
9	Geology
10	History
11	Materials science
12	Mathematics
13	Medicine
14	Philosophy
15	Physics
16	Political science
17	Psychology
18	Sociology`

The idea is if we can extract noun phrases from this dataset and give them a label. We can use a classifier to train on it and then classify our queries.

In [1]:
import jsonlines
import tqdm
import json
import spacy
import numpy as np 
import pandas as pd

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# import scispacy
import spacy
# import en_core_web_sm

from scipy.spatial.distance import jensenshannon

import joblib

from IPython.display import HTML, display

from ipywidgets import interact, Layout, HBox, VBox, Box
import ipywidgets as widgets
from IPython.display import clear_output

from tqdm import tqdm
from os.path import isfile

import seaborn as sb
import matplotlib.pyplot as plt
plt.style.use("dark_background")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs
from sklearn.model_selection import train_test_split
sns.set_style("whitegrid")
np.random.seed(0)

os.environ["CUDA_VISIBLE_DEVICES"]="1"

EMBEDDING_DIR = '/data/szr207/projects/log_analysis'

MAX_NB_WORDS = 100000
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])
# DATA_PATH = '/data/szr207/projects/log_analysis/equal_scidocs.csv'

from subprocess import check_output
# print(check_output(["ls", DATA_PATH]).decode("utf8"))
plt.style.use("dark_background")

# DATA_PATH = '/data/szr207/dataset/scidocs/mag/combined.csv'


/data/szr207/conda/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/szr207/conda/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/szr207/conda/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/szr207/conda/envs/tf-gpu/lib/python3.7/site-packages

In [2]:
path_json = '/data/szr207/dataset/scidocs/paper_metadata_mag_mesh.json'

with open(path_json) as f:
    paper_metadata_mag_mesh = json.load(f)

In [3]:
for i in list(paper_metadata_mag_mesh.keys())[:10]:
    print(i, paper_metadata_mag_mesh[i], paper_metadata_mag_mesh[i]['abstract'])

00014a8515491f0b3fe2a1ff6e0f5305e584dcd9 {'abstract': None, 'authors': ['115471397'], 'cited_by': [], 'paper_id': '00014a8515491f0b3fe2a1ff6e0f5305e584dcd9', 'references': [], 'title': 'ON THE CLASSIFICATION OF THE SCIENCES', 'year': None} None
00021eeee2bf4e06fec98941206f97083c38b54d {'abstract': 'Numerous studies of E-Commerce have emphasized the sway of obstacles related to Information Technology (IT) infrastructure, Socio economic condition, Internet usage, Government and regulatory initiatives as major determinants of ecommerce success. Information communication technology (ICT) revolutionizes businesses around the world. In this paper e-commerce practices in Malaysia has been analyzed by its strength, weaknesses, opportunities and threats. This paper provides a general review of challenges and opportunities for e commerce practices in Malaysia. And to understand the in-depth of e-commerce practices and to investigates more solutions for the research problems. Analysis shows that 

In [4]:
import spacy
# nlp = spacy.load('en_core_web_trf')
nlp = spacy.load("en_core_web_sm")

In [28]:
from collections import defaultdict

dict_idx_np = defaultdict(list)
# 
for i in tqdm(list(paper_metadata_mag_mesh.keys())):
#     print(i, paper_metadata_mag_mesh[i]['title'], paper_metadata_mag_mesh[i]['abstract'])
    if paper_metadata_mag_mesh[i]['title']:
#         doc = nlp(paper_metadata_mag_mesh[i]['title'])
#         for chunk in doc.noun_chunks:
#             dict_idx_np[i].append((chunk.text, 'np'))
        dict_idx_np[i].append((paper_metadata_mag_mesh[i]['title'], 'title'))
    if paper_metadata_mag_mesh[i]['abstract']:
        doc = nlp(paper_metadata_mag_mesh[i]['abstract'])
        for chunk in doc.noun_chunks:
            dict_idx_np[i].append((chunk.text, 'np'))

100%|██████████| 48473/48473 [18:19<00:00, 44.08it/s] 


In [29]:
json = json.dumps(dict_idx_np)
f = open("dict_idx_np_title_tag.json","w")
f.write(json)
f.close()
# import json

# # f = open('dict_idx_np.json',) 
# with open('../dict_idx_np.json') as f:
#     dict_idx_np = json.load(f)
# #     print(d)

AttributeError: 'str' object has no attribute 'dumps'

In [44]:
text = []
label = []

for idx in dict_idx_np:
#     print(idx)
    try:
        for x in dict_idx_np[idx]:
            text.append(x[0])
            label.append(x[1])
    except:
        continue

In [39]:
dict_idx_np['0014e12eac7d90bb1959b6b2bc4f2b481cb0a512'][0][1]

'title'

In [49]:
text = pd.Series(text)

In [50]:
label = pd.Series(label)

In [52]:
text.shape

(2284510,)

In [53]:
df_text = pd.Series(text,
              name="text")
df_text = df_text.to_frame()

df_label = pd.Series(label,
              name="label")
df_label = df_label.to_frame()

In [54]:
df_new_final = pd.concat([df_text, df_label], axis=1)

In [55]:
df_new_final.shape

(2284510, 2)

In [60]:
df_title = df_new_final[df_new_final['label']=='title']

In [61]:
df_np = df_new_final[df_new_final['label']=='np'].sample(n=48473 , random_state=7)

In [64]:
df = []
df.append(df_title)
df.append(df_np)

In [65]:
df

[                                                      text  label
 0                    ON THE CLASSIFICATION OF THE SCIENCES  title
 1        Opportunities and challenges for E-Commerce in...  title
 70       Interactions between model membranes and ligni...  title
 118            Perpetual Peace: What Kant Should Have Said  title
 119        Reimagining Greek Tragedy on the American Stage  title
 ...                                                    ...    ...
 2284164  Comparison of olmesartan combined with a calci...  title
 2284223  Role of angiotensin II in plasma PAI-1 changes...  title
 2284306  Factors that influence cancer patients' anxiet...  title
 2284367  Is adjuvant chemotherapy indicated in stage I ...  title
 2284456  Relationship between PCO2 and unfavorable outc...  title
 
 [48473 rows x 2 columns],                              text label
 1439011     the most common grade    np
 68616                   that task    np
 1886113                       who    np
 105

In [66]:
data = pd.concat(df, axis=0).reset_index(drop=True)

In [68]:
data.to_csv('noun_title_classification.csv', index=False)

In [72]:
data.sample(frac=1).to_csv('noun_title_classification.csv', index=False)

In [73]:
!head noun_title_classification.csv

text,label
they,np
PhD training in simulation: NATCOR,title
Effect of SLCO1B1 Polymorphisms on Rifabutin Pharmacokinetics in African HIV-Infected Patients with Tuberculosis.,title
methodology,np
A randomized controlled trial of the efficacy and safety of electroanatomic circumferential pulmonary vein ablation supplemented by ablation of complex fractionated atrial electrograms versus potential-guided pulmonary vein antrum isolation guided by intracardiac ultrasound.,title
"Hospitalizations among female home-helpers in Denmark, 1981-1997.",title
"Physical, Environmental, Social and Cultural Impacts of Climate Change on Europe's Regions",title
strong historical and cultural value,np
money,np
